# Getting Started

This notebook is interactive, and contains both code and text. To run a piece of code, click on the box, and then hit Shift-Enter. To edit a piece of text, just double click.

Let's start with a simple Regent example:

In [ ]:
-- Every Regent program must start with 'import "regent"'.
import "regent"

-- Regent is an embedded programming language. The top level of a file is a
-- Lua/Terra script. Certain keywords invoke the Regent (or Terra) compiler.
-- For example, this next block defines a Lua function, and calls it to
-- create a Terra struct.

function pointof(t)
  return struct {
    x : t,
    y : t,
    z : t,
  }
end
local point = pointof(int)

-- Tasks can be declared with the 'task' keyword. Tasks are like functions,
-- but must explicitly declare *privileges* ('reads', 'writes', etc.) on
-- their arguments. Let's ignore the body of the task for the moment (we'll
-- come back to it in a moment).

task inc_x(points : region(point), amount : int)
where reads writes(points.x) do
  for pt in points do
    pt.x += amount
  end
end

-- Typically, execution starts at a main task. Main is just a task which
-- takes no arguments.

task main()
  -- Conceptually, tasks can be thought to execute sequentially. So we can
  -- read the code in each task top-to-bottom, like a normal sequential
  -- program.

  -- Let's start by creating regions. Regions are like containers, similar
  -- to arrays of structs. Regions are indexed by a index space (in this
  -- case, it just contains 5 elements). Each element is of the type
  -- specified by the field space (in this case, point---so each element
  -- will have fields x, y, and z).
  var points = region(ispace(ptr, 5), point)

  -- Regions are empty by default. We need to allocate some elements inside
  -- the region before we can do anything with the contents.
  var point0 = new(ptr(point, points))
  var point1 = new(ptr(point, points))
  var point2 = new(ptr(point, points))
  var point3 = new(ptr(point, points))
  var point4 = new(ptr(point, points))

  -- We can initialize the region just by writing a for loop over the
  -- contents.
  for pt in points do
    pt.x = 10
  end

  -- We can also pass regions as arguments to tasks, like the one we
  -- defined above.
  inc_x(points, 200)

  -- If we want to harness data parallelism, we'll need to partition our
  -- region into subregions. To do this, we'll need to color the elements of
  -- the region according to which subpiece they should belong to.
  var i = 0
  for pt in points do
    pt.z = i/2 -- Assign each point a color.
    i += 1
  end
  var colors = ispace(index_type(int), 3) -- Set of colors in partition.
  var subpoints = partition(points.z, colors) -- Create the partition.

  -- Now we can call tasks on each of the subregions.
  for color in colors do
    inc_x(subpoints[int(color)], 3000)
  end

  i = 0
  for pt in points do
    regentlib.c.printf("point %d has value %d\n", i, pt.x)
    regentlib.assert(pt.x == 3210, "test failed")
    i += 1
  end
end
regentlib.start(main)

## SAXPY

Here's another example Regent program. This one computes SAXPY ($aX + Y$). Notice that, unlike the code above which used the generic opaque `ptr` type for indexing, this code uses array-style indexing with `int`s.

In [ ]:
import "regent"

-- Terra makes it easy to include C headers. In this case, we're going to
-- include 'math.h' so that we can call fabs.
local c = terralib.includec("math.h")
local abs = c.fabs

-- Remember the 'ptr' type from earlier? That was an index type representing
-- an opaque pointer. We can make our own index types too. Here, we're going
-- to create an index type based on a base type of 'int'. This new index type
-- won't be opaque, so we'll be able to do index arithmetic on the indices.
local int1d = index_type(int, "int1d")

-- Tasks can take index spaces as arguments as well as regions. In this case,
-- we want to use a single index space with both 'x' and 'y' so that the
-- compiler knows the indices are valid in both regions.
task saxpy(is : ispace(int1d), x: region(is, float), y: region(is, float), a: float)
where
  reads(x, y), writes(y)
do
  for i in is do
    y[i] += a*x[i]
  end
end

task main()
  var n = 10
  var is = ispace(int1d, n)
  var x = region(is, float)
  var y = region(is, float)

  for i in is do
    x[i] = 1.0
    y[i] = 0.0
  end

  saxpy(is, x, y, 0.5)

  for i in is do
    regentlib.c.printf("%f\n", y[i])
    regentlib.assert(abs(y[i] - 0.5) < 0.00001, "test failed")
  end
end
regentlib.start(main)